#循环遍历每一集

###旧版本，append 方法在 pandas 1.4.0 及以后的版本中已被弃用，建议改用 concat 方法。我们可以通过构建一个临时的 DataFrame 列表，然后一次性将其合并。

In [ ]:
import pandas as pd

# 初始化一个空 DataFrame 用于存储所有结果
all_correlation_df = pd.DataFrame(columns=['episode', 'source', 'target', 'correlation'])

# 遍历循环mapped_info_episode_1.xlsx到mapped_info_episode_10.xlsx
for i in range(1, 17):
    # 读取 mapped_info_episode_i.xlsx 文件
    #mapped_info_df = pd.read_excel(f'D:/研究生论文/小论文2/代码/02质量特征识别/02质量特征匹配结果/无限世界4.8_每集/mapped_info_episode_{i}.xlsx')
    mapped_info_df = pd.read_excel(f'D:/研究生论文/小论文2/代码/02RAG_质量特征匹配阶段/RAG代码/01输出数据/无限世界_每集/mapped_info_episode_{i}.xlsx')

    # 读取 network_edges_episode_i.xlsx 文件
    network_edges_df = pd.read_excel(f'D:/研究生论文/小论文2/代码/03网络图/01网络图构建/边数据/无限世界RAG/network_edges_episode_{i}.xlsx')

    # 提取 mapped_info_df 中的特征和值
    features = mapped_info_df[['Feature_1', 'Feature_2', 'Feature_3']].values
    values = mapped_info_df[['Value_1', 'Value_2', 'Value_3']].values

    # 提取 network_edges_df 中的源和目标
    sources = network_edges_df['source'].values
    targets = network_edges_df['target'].values

    # 初始化一个空 DataFrame 用于存储相关系数
    correlation_df = pd.DataFrame(columns=['episode', 'source', 'target', 'correlation'])

    # 遍历每对源和目标
    for source, target in zip(sources, targets):
        # 判断特征中是否包含源和目标
        if source in features and target in features:
            # 获取对应的值
            source_values = values[features == source].flatten()
            target_values = values[features == target].flatten()

            # 计算相关系数
            correlation = pd.Series(source_values).corr(pd.Series(target_values))

            # 将结果添加到 DataFrame 中
            correlation_df = correlation_df.append({'episode': i, 'source': source, 'target': target, 'correlation': correlation}, ignore_index=True)

    # 将当前结果添加到所有结果中
    all_correlation_df = pd.concat([all_correlation_df, correlation_df])

# 保存所有结果到一个 Excel 文件中
all_correlation_df.to_excel('D:/研究生论文/小论文2/代码/03网络图/02节点之间关系/无限世界RAG_all_correlation_results.xlsx', index=False)


###新版本--可用

In [2]:
import pandas as pd

# 初始化一个空 DataFrame 用于存储所有结果
all_correlation_df = pd.DataFrame(columns=['episode', 'source', 'target', 'correlation'])

# 遍历循环 mapped_info_episode_1.xlsx 到 mapped_info_episode_16.xlsx
for i in range(1, 92):
    # 读取 mapped_info_episode_i.xlsx 文件
    mapped_info_df = pd.read_excel(f'D:/研究生论文/小论文2/代码/02RAG_质量特征匹配阶段/RAG代码/01输出数据/桃花马上请长缨_每集/mapped_info_episode_{i}.xlsx')

    # 读取 network_edges_episode_i.xlsx 文件
    network_edges_df = pd.read_excel(f'D:/研究生论文/小论文2/代码/03网络图/01网络图构建/边数据/桃花马上请长缨RAG/network_edges_episode_{i}.xlsx')

    # 初始化一个列表用于存储相关系数
    correlation_list = []

    # 遍历每对源和目标
    for source, target in zip(network_edges_df['source'], network_edges_df['target']):
        # 获取源的值
        source_values = mapped_info_df.loc[
            (mapped_info_df['Feature_1'] == source) | 
            (mapped_info_df['Feature_2'] == source) | 
            (mapped_info_df['Feature_3'] == source), 
            ['Value_1', 'Value_2', 'Value_3']
        ].values.flatten()

        # 获取目标的值
        target_values = mapped_info_df.loc[
            (mapped_info_df['Feature_1'] == target) | 
            (mapped_info_df['Feature_2'] == target) | 
            (mapped_info_df['Feature_3'] == target), 
            ['Value_1', 'Value_2', 'Value_3']
        ].values.flatten()

        # 将值转换为数值类型，移除无效值
        source_values = pd.to_numeric(source_values, errors='coerce')
        target_values = pd.to_numeric(target_values, errors='coerce')

        # 确保源和目标值的长度一致
        if len(source_values) != len(target_values):
            min_length = min(len(source_values), len(target_values))
            source_values = source_values[:min_length]
            target_values = target_values[:min_length]

        # 移除无效值
        valid_indices = ~pd.isnull(source_values) & ~pd.isnull(target_values)
        source_values = source_values[valid_indices]
        target_values = target_values[valid_indices]

        # 检查是否有足够的有效数据点计算相关系数
        if len(source_values) > 1 and len(target_values) > 1:
            # 计算相关系数
            correlation = pd.Series(source_values).corr(pd.Series(target_values))

            # 将结果添加到列表中
            correlation_list.append({'episode': i, 'source': source, 'target': target, 'correlation': correlation})

    # 将当前结果添加到所有结果中
    if correlation_list:
        correlation_df = pd.DataFrame(correlation_list)
        all_correlation_df = pd.concat([all_correlation_df, correlation_df], ignore_index=True)

# 保存所有结果到一个 Excel 文件中
output_file_path = 'D:/研究生论文/小论文2/代码/03网络图/02节点之间关系/桃花马上请长缨RAG_all_correlation_results.xlsx'
all_correlation_df.to_excel(output_file_path, index=False)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_28484\3212893193.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_correlation_df = pd.concat([all_correlation_df, correlation_df], ignore_index=True)
d:\software\anaconda\anaconda\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
d:\software\anaconda\anaconda\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
d:\software\anaconda\anaconda\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
d:\software\anaconda\anaconda\Lib\site-packages\numpy\lib\function_base.py:2897

##将结果根据集数拆分

In [3]:
import pandas as pd

# 读取包含所有数据的 Excel 文件
all_correlation_df = pd.read_excel('D:/研究生论文/小论文2/代码/03网络图/02节点之间关系/桃花马上请长缨RAG_all_correlation_results.xlsx')

# 获取所有唯一的集数
episodes = all_correlation_df['episode'].unique()

# 遍历每个集数，并将其数据保存到单独的 Excel 文件中
for episode in episodes:
    # 提取当前集数的数据
    episode_data = all_correlation_df[all_correlation_df['episode'] == episode]
    
    # 构造保存文件的路径
    filename = f'D:/研究生论文/小论文2/代码/03网络图/02节点之间关系/桃花马上请长缨RAG/correlation_results_episode_{episode}.xlsx'
    
    # 保存数据到 Excel 文件
    episode_data.to_excel(filename, index=False)

print("拆分完成！")


拆分完成！


##每一集的

In [ ]:

import pandas as pd

# 读取 mapped_info_episode_1.xlsx 文件
mapped_info_df = pd.read_excel('E:/桌面/代码/02质量特征识别/02质量特征匹配结果/百妖谱_每集/mapped_info_episode_1.xlsx')

# 读取 network_edges_episode_1.xlsx 文件
network_edges_df = pd.read_excel('E:/桌面/代码/03网络图/01网络图构建/边数据/百妖谱/network_edges_episode_1.xlsx')

# 提取 mapped_info_df 中的特征和值
features = mapped_info_df[['Feature_1', 'Feature_2', 'Feature_3']].values
values = mapped_info_df[['Value_1', 'Value_2', 'Value_3']].values

# 提取 network_edges_df 中的源和目标
sources = network_edges_df['source'].values
targets = network_edges_df['target'].values

# 初始化一个空 DataFrame 用于存储相关系数
correlation_df = pd.DataFrame(columns=['source', 'target', 'correlation'])

# 遍历每对源和目标
for source, target in zip(sources, targets):
    # 判断特征中是否包含源和目标
    if source in features and target in features:
        # 获取对应的值
        source_values = values[features == source].flatten()
        target_values = values[features == target].flatten()
        
        # 计算相关系数
        correlation = pd.Series(source_values).corr(pd.Series(target_values))
        
        # 将结果添加到 DataFrame 中
        correlation_df = correlation_df.append({'source': source, 'target': target, 'correlation': correlation}, ignore_index=True)

# 保存结果到 Excel 文件中
correlation_df.to_excel('E:/桌面/代码/03网络图/02节点之间关系/百妖谱/correlation_results.xlsx', index=False)
